In [47]:
import numpy as np
from distributed import Client
import dask.dataframe as dd
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, StringType, IntegerType

In [2]:
client = Client(n_workers=12, threads_per_worker=2, processes=False, memory_limit='4GB')
client

Client Scheduler: inproc://192.168.1.253/38408/1 Dashboard: http://192.168.1.253:8787/status,Cluster Workers: 12 Cores: 24 Memory: 48.00 GB


In [3]:
orig_helper = dd.read_csv('resources/orig_read_helper.csv').compute().set_index('col_idx')
orig_dtypes = orig_helper['dtype'].to_dict()
orig_col_names = orig_helper['name'].to_dict()

orig = dd.read_csv('sample_orig_2009.txt', sep='|', header=None, dtype=orig_dtypes)\
         .compute().rename(columns=orig_col_names)
orig.head()

,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,...,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,floaterest Only (I/O) Indicator
0,795.0,200903.0,N,202402.0,NaN,0.0,1.0,P,60.0,12.0,...,180.0,2.0,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,9.0,N
1,766.0,200903.0,Y,203902.0,24020.0,0.0,1.0,P,80.0,31.0,...,360.0,1.0,Other sellers,Other servicers,NaN,NaN,9,NaN,9.0,N
2,701.0,200903.0,N,203902.0,13380.0,0.0,1.0,P,74.0,61.0,...,360.0,1.0,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,9.0,N
3,791.0,200903.0,N,203902.0,36540.0,0.0,1.0,P,64.0,38.0,...,360.0,1.0,Other sellers,Other servicers,NaN,NaN,9,NaN,9.0,N
4,725.0,200903.0,N,203902.0,36540.0,0.0,1.0,P,85.0,29.0,...,360.0,2.0,Other sellers,Other servicers,NaN,NaN,9,NaN,9.0,N


In [138]:
svcg_helper

,name,dtype,length
col_idx,,,
0,Loan Sequence Number,str,12
1,Monthly Reporting Period,float32,6
2,Current Actual UPB,float64,12
3,Current Loan Delinquency Status,str,3
4,Loan Age,float16,3
5,Remaining Months to Legal Maturity,float16,3
6,Repurchase Flag,str,1
7,Modification Flag,str,1
8,Zero Balance Code,float16,2


In [ ]:
.toPandas().set_index('name').T.to_dict('list')

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, StringType, IntegerType

# start a spark session
spark = SparkSession \
    .builder \
    .appName("label generation") \
    .getOrCreate()

# read csv to specify schema for reading svcg files
svcg_helper = spark.read.csv('resources/svcg_read_helper.csv', header=True)
# transform to Pandas format to generate a dictionary 
# since the helper only aims to specify a fixed set of schema for each column, it does not need be scaled
svcg_helper = svcg_helper.toPandas().set_index('col_idx').to_dict()

# create schema for reading svcg file
struct_field_lst = []
for i in range(len(svcg_helper['name'])):
    name = svcg_helper['name'][f'{i}']
    dtype = FloatType() if ('float' in svcg_helper['dtype'][f'{i}']) else StringType()
    # connect column name with dtype
    struct_field_lst.append(StructField(name, dtype, True))
schema = StructType(struct_field_lst)

# create dataframe
svcg = spark.read.csv('sample_svcg_2009.txt', sep='|', schema=schema)

# generate labels with specified conditions
svcg = svcg.withColumn('Label',  
    # delinquency status greater than 3 means above 90 days of delinquency 
    ((svcg['Current Loan Delinquency Status'].isNotNull()) & (svcg['Current Loan Delinquency Status'] >= 3) & 
    # filter out REO Acquisition, Unknown, and Unaviable options
     (~(svcg['Current Loan Delinquency Status']).isin(['R', 'XX', '   ']))) | 
    # select those who have zero balance code of 3, 6, 9
    ((svcg['Zero Balance Code'].isNotNull()) & (svcg['Zero Balance Code'].isin([3.0, 6.0, 9.0])))
                      )

# cast label as integer type
svcg = svcg.withColumn("Label", svcg["Label"].cast(IntegerType()))

# rename the column name to be legal for parquet format
svcg = svcg.withColumnRenamed("Loan Sequence Number", "Loan_Sequence_Number")

# output parquet file with primary key and label
svcg.select(['Loan_Sequence_Number','Label']).write.parquet("output/proto.parquet")

Py4JJavaError: An error occurred while calling o574.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:736)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:271)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:287)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:865)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:547)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:587)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:559)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:586)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.j

In [151]:
# read csv to specify schema for reading svcg files
svcg_helper = spark.read.csv('resources/svcg_read_helper.csv', header=True)
# transform to Pandas format to generate a dictionary 
# since the helper only aims to specify a fixed set of schema for each column, it does not need be scaled
svcg_helper = svcg_helper.toPandas().set_index('col_idx').to_dict()
svcg_helper

{'name': {'0': 'Loan Sequence Number',
  '1': 'Monthly Reporting Period',
  '2': 'Current Actual UPB',
  '3': 'Current Loan Delinquency Status',
  '4': 'Loan Age',
  '5': 'Remaining Months to Legal Maturity',
  '6': 'Repurchase Flag',
  '7': 'Modification Flag',
  '8': 'Zero Balance Code',
  '9': 'Zero Balance Effective Date',
  '10': 'Current floaterest Rate',
  '11': 'Current Deferred UPB',
  '12': 'Due Date of Last Paid Installment (DDLPI)',
  '13': 'MI Recoveries',
  '14': 'Net Sales Proceeds',
  '15': 'Non MI Recoveries',
  '16': 'Expenses',
  '17': 'Legal Costs',
  '18': 'Mafloatenance and Preservation Costs',
  '19': 'Taxes and Insurance',
  '20': 'Miscellaneous Expenses',
  '21': 'Actual Loss Calculation',
  '22': 'Modification Cost',
  '23': 'Step Modification Flag',
  '24': 'Deferred Payment Plan',
  '25': 'Estimated Loan-to-Value (ELTV)',
  '26': 'Zero Balance Removal UPB',
  '27': 'Delinquent Accrued floaterest',
  '28': 'Delinquency Due to Disaster',
  '29': 'Borrower Assi

In [137]:
svcg_helper = dd.read_csv('resources/svcg_read_helper.csv').compute().set_index('col_idx')
svcg_dtypes = svcg_helper['dtype'].to_dict()
svcg_col_names = svcg_helper['name'].to_dict()

svcg = dd.read_csv('sample_svcg_2009.txt', sep='|', header=None, dtype=svcg_dtypes)\
         .compute().rename(columns=svcg_col_names)
svcg.head(30)

,Loan Sequence Number,Monthly Reporting Period,Current Actual UPB,Current Loan Delinquency Status,Loan Age,Remaining Months to Legal Maturity,Repurchase Flag,Modification Flag,Zero Balance Code,Zero Balance Effective Date,...,Miscellaneous Expenses,Actual Loss Calculation,Modification Cost,Step Modification Flag,Deferred Payment Plan,Estimated Loan-to-Value (ELTV),Zero Balance Removal UPB,Delinquent Accrued floaterest,Delinquency Due to Disaster,Borrower Assistance Status Code
0,F09Q10000013,200902.0,157000.00,0,0.0,180.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F09Q10000013,200903.0,156000.00,0,1.0,179.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F09Q10000013,200904.0,155000.00,0,2.0,178.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F09Q10000013,200905.0,155000.00,0,3.0,177.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,F09Q10000013,200906.0,154000.00,0,4.0,176.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,F09Q10000013,200907.0,153000.00,0,5.0,175.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,F09Q10000013,200908.0,152000.00,0,6.0,174.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,F09Q10000013,200909.0,151743.42,0,7.0,173.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,F09Q10000013,200910.0,151042.83,0,8.0,172.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,F09Q10000013,200911.0,150339.61,0,9.0,171.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
svcg_helper

{'name': {0: 'Loan Sequence Number',
  1: 'Monthly Reporting Period',
  2: 'Current Actual UPB',
  3: 'Current Loan Delinquency Status',
  4: 'Loan Age',
  5: 'Remaining Months to Legal Maturity',
  6: 'Repurchase Flag',
  7: 'Modification Flag',
  8: 'Zero Balance Code',
  9: 'Zero Balance Effective Date',
  10: 'Current floaterest Rate',
  11: 'Current Deferred UPB',
  12: 'Due Date of Last Paid Installment (DDLPI)',
  13: 'MI Recoveries',
  14: 'Net Sales Proceeds',
  15: 'Non MI Recoveries',
  16: 'Expenses',
  17: 'Legal Costs',
  18: 'Mafloatenance and Preservation Costs',
  19: 'Taxes and Insurance',
  20: 'Miscellaneous Expenses',
  21: 'Actual Loss Calculation',
  22: 'Modification Cost',
  23: 'Step Modification Flag',
  24: 'Deferred Payment Plan',
  25: 'Estimated Loan-to-Value (ELTV)',
  26: 'Zero Balance Removal UPB',
  27: 'Delinquent Accrued floaterest',
  28: 'Delinquency Due to Disaster',
  29: 'Borrower Assistance Status Code'},
 'dtype': {0: 'str',
  1: 'float32',
 

In [116]:
spark = SparkSession \
    .builder \
    .appName("label generation") \
    .getOrCreate()

In [141]:
svcg_helper = svcg_helper.to_dict()
struct_field_lst = []
for i in range(len(svcg_helper['name'])):
    name = svcg_helper['name'][i]
    dtype = FloatType() if ('float' in svcg_helper['dtype'][i]) else StringType()
    struct_field_lst.append(StructField(name, dtype, True))
schema = StructType(struct_field_lst)
schema

StructType(List(StructField(Loan Sequence Number,StringType,true),StructField(Monthly Reporting Period,FloatType,true),StructField(Current Actual UPB,FloatType,true),StructField(Current Loan Delinquency Status,StringType,true),StructField(Loan Age,FloatType,true),StructField(Remaining Months to Legal Maturity,FloatType,true),StructField(Repurchase Flag,StringType,true),StructField(Modification Flag,StringType,true),StructField(Zero Balance Code,FloatType,true),StructField(Zero Balance Effective Date,FloatType,true),StructField(Current floaterest Rate,FloatType,true),StructField(Current Deferred UPB,FloatType,true),StructField(Due Date of Last Paid Installment (DDLPI),FloatType,true),StructField(MI Recoveries,FloatType,true),StructField(Net Sales Proceeds,StringType,true),StructField(Non MI Recoveries,FloatType,true),StructField(Expenses,FloatType,true),StructField(Legal Costs,FloatType,true),StructField(Mafloatenance and Preservation Costs,FloatType,true),StructField(Taxes and Insuranc

In [129]:


# create dataframe
svcg = spark.read.csv('sample_svcg_2009.txt', sep='|', schema=schema)

svcg = svcg.withColumn('Label',  
    # delinquency status greater than 3 means above 90 days of delinquency 
    ((svcg['Current Loan Delinquency Status'].isNotNull()) & (svcg['Current Loan Delinquency Status'] >= 3) & 
    # filter out REO Acquisition, Unknown, and Unaviable options
     (~(svcg['Current Loan Delinquency Status']).isin(['R', 'XX', '   ']))) | 
    # select those who have zero balance code of 3, 6, 9
    ((svcg['Zero Balance Code'].isNotNull()) & (svcg['Zero Balance Code'].isin([3.0, 6.0, 9.0])))
                      )

In [130]:
svcg = svcg.withColumn("Label", svcg["Label"].cast(IntegerType()))

In [131]:
svcg.groupBy('Label').count().show()

+-----+-------+
|Label|  count|
+-----+-------+
|    1|  17727|
|    0|2934219|
+-----+-------+



In [86]:
svcg.where(svcg.Label==1).show()

+--------------------+------------------------+------------------+-------------------------------+--------+----------------------------------+---------------+-----------------+-----------------+---------------------------+-----------------------+--------------------+-----------------------------------------+-------------+------------------+-----------------+---------+-----------+------------------------------------+-------------------+----------------------+-----------------------+-----------------+----------------------+---------------------+------------------------------+------------------------+-----------------------------+---------------------------+-------------------------------+-----+
|Loan Sequence Number|Monthly Reporting Period|Current Actual UPB|Current Loan Delinquency Status|Loan Age|Remaining Months to Legal Maturity|Repurchase Flag|Modification Flag|Zero Balance Code|Zero Balance Effective Date|Current floaterest Rate|Current Deferred UPB|Due Date of Last Paid Installme

In [83]:
svcg.where(col("Label").isNull())

NameError: name 'col' is not defined

In [34]:
svcg['Zero Balance Code'].isNotNull().groupBy($"value").count.orderBy($"count".desc)

SyntaxError: invalid syntax (<ipython-input-34-24ad08d9def5>, line 1)

In [37]:
svcg['Zero Balance Code'].isNotNull().groupBy('Zero Balance Code').count().orderBy('count').show()

TypeError: 'Column' object is not callable

In [27]:
svcg.select('Zero Balance Code').show()

+-----------------+
|Zero Balance Code|
+-----------------+
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
|             null|
+-----------------+
only showing top 20 rows



In [46]:
svcg['Loan Sequence Number']

Column<'Loan Sequence Number'>

In [29]:
schema

StructType(List(StructField(member_srl,IntegerType,true),StructField(click_day,IntegerType,true),StructField(productid,IntegerType,true)))

In [26]:
svcg_helper.shape

(30, 3)

In [22]:
svcg_helper.to_dict()

{'name': {0: 'Loan Sequence Number',
  1: 'Monthly Reporting Period',
  2: 'Current Actual UPB',
  3: 'Current Loan Delinquency Status',
  4: 'Loan Age',
  5: 'Remaining Months to Legal Maturity',
  6: 'Repurchase Flag',
  7: 'Modification Flag',
  8: 'Zero Balance Code',
  9: 'Zero Balance Effective Date',
  10: 'Current floaterest Rate',
  11: 'Current Deferred UPB',
  12: 'Due Date of Last Paid Installment (DDLPI)',
  13: 'MI Recoveries',
  14: 'Net Sales Proceeds',
  15: 'Non MI Recoveries',
  16: 'Expenses',
  17: 'Legal Costs',
  18: 'Mafloatenance and Preservation Costs',
  19: 'Taxes and Insurance',
  20: 'Miscellaneous Expenses',
  21: 'Actual Loss Calculation',
  22: 'Modification Cost',
  23: 'Step Modification Flag',
  24: 'Deferred Payment Plan',
  25: 'Estimated Loan-to-Value (ELTV)',
  26: 'Zero Balance Removal UPB',
  27: 'Delinquent Accrued floaterest',
  28: 'Delinquency Due to Disaster',
  29: 'Borrower Assistance Status Code'},
 'dtype': {0: 'str',
  1: 'float32',
 

In [13]:
spark.read.csv?

In [15]:
svcg.head()

Row(_c0='F09Q10000013', _c1='200902', _c2='157000.00', _c3='0', _c4='000', _c5='180', _c6=None, _c7=None, _c8=None, _c9=None, _c10='4.500', _c11='0.00', _c12=None, _c13=None, _c14=None, _c15=None, _c16=None, _c17=None, _c18=None, _c19=None, _c20=None, _c21=None, _c22=None, _c23=None, _c24=None, _c25=None, _c26=None, _c27=None, _c28=None, _c29=None)

In [101]:
spark.read.format("csv").load("sample_svcg_2009.txt")

DataFrame[_c0: string]

In [104]:
svcg['Zero Balance Code'].value_counts()

1.0     43629
9.0       221
15.0      150
6.0       150
3.0       122
2.0        96
Name: Zero Balance Code, dtype: int64

In [107]:
svcg['Current Loan Delinquency Status'].value_counts().head(50)

0     2906738
1       21370
2        6015
3        2708
4        1990
5        1653
6        1370
7        1080
8         936
9         823
10        698
11        580
12        518
13        473
R         443
14        384
15        365
16        319
17        290
18        262
19        240
20        230
21        193
22        175
23        153
24        150
25        125
27        120
26        118
28        104
29        100
30         91
31         80
32         77
33         74
35         69
34         68
36         63
38         57
37         57
39         49
40         48
41         38
43         36
42         32
44         27
45         24
49         22
46         22
47         21
Name: Current Loan Delinquency Status, dtype: int64

In [48]:
pd.Int64Dtype()

Int64Dtype()

In [29]:
df = pd.read_csv('sample_orig_2009.txt', sep='|', header=None)
df.iloc[0]

0                795
1             200903
2                  N
3             202402
4                NaN
5                  0
6                  1
7                  P
8                 60
9                 12
10            157000
11                60
12               4.5
13                 R
14                 N
15               FRM
16                MN
17                SF
18             56600
19      F09Q10000013
20                 C
21               180
22                 2
23     Other sellers
24    U.S. BANK N.A.
25               NaN
26               NaN
27                 9
28               NaN
29                 9
30                 N
Name: 0, dtype: object

In [5]:
dd.read_csv?

In [ ]:
dtypes_orig = {0: 'int16', # credit score 4 digits
               1: 'int32', # first payment date YYYYMM
               2: 'U1', # first time homebuyer flag Y or N
               3: 'int32', # matruity
              }

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,object,int64,float64,int64,int64,object,int64,int64,int64,int64,float64,object,object,object,object,object,int64,object,object,int64,int64,object,object,object,object,int64,object,int64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [23]:
pd.read_csv('sample_orig_2009.txt', sep='|', header=None)

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,795,200903,N,202402,NaN,0,1,P,60,12,...,180,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,9,N
1,766,200903,Y,203902,24020.0,0,1,P,80,31,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N
2,701,200903,N,203902,13380.0,0,1,P,74,61,...,360,1,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,9,N
3,791,200903,N,203902,36540.0,0,1,P,64,38,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N
4,725,200903,N,203902,36540.0,0,1,P,85,29,...,360,2,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,749,201001,Y,203912,19060.0,0,1,P,100,43,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N
49996,709,200912,Y,203911,16740.0,0,1,P,100,40,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N
49997,801,201001,Y,203912,28940.0,0,1,P,100,36,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N
49998,705,201002,Y,204001,NaN,0,1,P,93,34,...,360,2,Other sellers,Other servicers,NaN,NaN,9,NaN,9,N


In [18]:
dd.read_csv('historical_data_time_2009Q1.txt', sep='|', header=None, low_memory=False).head(n=20)

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.40 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.40 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.41 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.41 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.41 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARN

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------+--------+----------+
| Column | Found  | Expected |
+--------+--------+----------+
| 24     | object | float64  |
| 28     | object | float64  |
| 29     | object | float64  |
| 3      | object | int64    |
+--------+--------+----------+

The following columns also raised exceptions on conversion:

- 24
  ValueError("could not convert string to float: 'Y'")
- 28
  ValueError("could not convert string to float: 'Y'")
- 29
  ValueError("could not convert string to float: 'F'")
- 3
  ValueError("invalid literal for int() with base 10: 'R'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={24: 'object',
       28: 'object',
       29: 'object',
       3: 'object'}

to the call to `read_csv`/`read_table`.

In [110]:
temp = pd.read_csv('sample_svcg_2009.txt', sep='|', header=None)

C:\Users\ziw029\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,7,23,24,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [114]:
temp[3].value_counts()

0      2579903
0       326835
1        19452
2         5566
3         2503
1         1918
4         1842
5         1534
6         1271
7          989
8          869
9          763
10         642
11         541
12         481
2          449
R          443
13         435
14         351
15         332
16         292
17         262
18         240
19         217
20         211
3          205
21         177
22         161
4          148
23         144
24         138
5          119
25         119
26         112
27         106
28         100
6           99
29          96
7           91
30          87
31          76
32          73
33          70
8           67
35          65
34          64
9           60
36          60
10          56
37          55
38          55
39          47
40          46
11          39
13          38
12          37
41          36
43          34
14          33
15          33
42          30
17          28
16          27
44          25
19          23
18          22
45        

In [113]:
pd.set_option('display.max_rows', 500)
